### MACHINE LEARNING PROJECT I

#### TEAM MEMBERS: ASLI YORUSUN - ERDEM BOCUGOZ - SERIF SONER SERBEST

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from helpers import *
from implementations import *
from data import *

%load_ext autoreload
%autoreload 2

### Importing Raw Data

In [2]:
prediction,data,id_ = load_csv_data("train.csv", sub_sample=False)

In [ ]:
data.shape

### Data Processing

In [25]:
pred_0, pred_1, pred_2, data_0, data_1, data_2, indices_0, indices_1, indices_2 = categorize_data(prediction, data)

In [3]:
x, mean_x, std_x = standardize(data)
y, tx = build_model_data(prediction, x)

In [26]:
#x, mean_x, std_x = process_data(data)
y, tx = build_model_data(prediction, data)

x0, mean_x0, std_x0 = process_data(data_0)
y0, tx0 = build_model_data(pred_0,x0)

x1, mean_x1, std_x1 = process_data(data_1)
y1, tx1 = build_model_data(pred_1,x1)

x2, mean_x2, std_x2 = process_data(data_2)
y2, tx2 = build_model_data(pred_2,x2)

In [ ]:
tx.shape

### Gradient Descent

In [4]:
# Define the parameters of the algorithm.
max_iters = 500
gamma = 0.1

# Initialization
initial_w = np.zeros(tx.shape[1])

weights, loss = least_squares_GD(y, tx, initial_w, max_iters, gamma)

Gradient Descent: execution time= 7.898 seconds
Gradient Descent: RMSE Loss = 0.4123272067563234


In [5]:
ypred = predict_labels(weights,tx)
ypred = np.where(ypred == -1, 0, ypred)
ypred = np.squeeze(ypred)

(ypred == y).mean()

0.74434

### Stochastic Gradient Descent

In [6]:
# Define the parameters of the algorithm.
max_iters = 500
gamma = 0.1

# Initialization
initial_w = np.zeros(tx.shape[1])

weights, loss = least_squares_GD(y, tx, initial_w, max_iters, gamma)

Gradient Descent: execution time= 8.513 seconds
Gradient Descent: RMSE Loss = 0.4123272067563234


In [9]:
ypred = predict_labels(weights,tx)
ypred = np.where(ypred == -1, 0, ypred)
ypred = np.squeeze(ypred)

(ypred == y).mean()

0.74497199999999997

### Least Squares

In [8]:
weights, loss = least_squares(y, tx)

Least Squares: execution time= 0.086 seconds
Least Squares: RMSE Loss = 0.41197427012456383


### Ridge Regression

In [ ]:
seed = 1
degree = 10
split_ratio = 0.9
lambdas = np.logspace(-10, 0, 50)

lambda_ = select_hyperparameter_for_ridge_regression(x, y, degree, split_ratio, seed, lambdas)

In [10]:
tx_poly = build_poly(x,7)
lambda_ = 0.001

weight,loss = ridge_regression(y,tx_poly,0.0001)

Ridge Regression: RMSE Loss = 0.37604574620474157


In [11]:
ypred = predict_labels(weight,tx_poly)
ypred = np.where(ypred == -1, 0, ypred)
ypred = np.squeeze(ypred)

(ypred == y).mean()

0.80797200000000002

### Cross Validation

In [ ]:
seed = 32
degree = 2
k_fold = 5
step = 3
lambdas = np.logspace(-10, 0, step)

# Cross Validation for Ridge Regression
lambda_ = select_hyperparameter_with_cross_validation(y, x, seed, degree, k_fold, step, lambdas)
weights, loss = ridge_regression(y, tx, lambda_)

#### Cross Validation for Subsets

In [ ]:
seed = 10
degree = 4
k_fold = 10
step = 20
lambdas = np.logspace(-3, 0, step)

# Cross Validation for Ridge Regression
#lambda_0 = select_hyperparameter_with_cross_validation(y0, x0, seed, degree, k_fold, step, lambdas)

poly_tx0_rid = build_poly(x0, degree)
w0, loss0 = ridge_regression(y0, poly_tx0_rid, 0.001)

In [ ]:
seed = 10
degree = 12
k_fold = 10
step = 50
lambdas = np.logspace(-10, 0, step)

# Cross Validation for Ridge Regression
#lambda_1 = select_hyperparameter_with_cross_validation(y1, x1, seed, degree, k_fold, step, lambdas)

poly_tx1_rid = (build_poly(x1, degree))
w1, loss1 = ridge_regression(y1, poly_tx1_rid, 0.00016)

In [ ]:
lambda_1

In [ ]:
seed = 10
degree = 12
k_fold = 10
step = 50
lambdas = np.logspace(-10, 0, step)

# Cross Validation for Ridge Regression
#lambda_2 = select_hyperparameter_with_cross_validation(y2, x2, seed, degree, k_fold, step, lambdas)

poly_tx2_rid =build_poly(x2, degree)
w2, loss2 = ridge_regression(y2, poly_tx2_rid, 7.91e-7)

In [ ]:
np.save("w0",w0)
tx_pp = poly_tx0

In [ ]:
y_pred0 =predict_labels(w0,tx_pp)
y_pred1 = predict_labels(w1,poly_tx1_rid)
y_pred2 = predict_labels(w2,poly_tx2_rid)

In [ ]:
y_pred0 = np.where(y_pred0 == -1, 0, y_pred0)
y_pred1 = np.where(y_pred1 == -1, 0, y_pred1)
y_pred2 = np.where(y_pred2 == -1, 0, y_pred2)

y_pred1 = np.expand_dims(y_pred1, axis=1)
y_pred2 = np.expand_dims(y_pred2, axis=1)

In [ ]:
y1.shape

In [ ]:

score0 = (np.squeeze(y_pred0) == y0).mean()
score1 = (np.squeeze(y_pred1) == y1).mean()
score2 = (np.squeeze(y_pred2) == y2).mean()



In [ ]:
print(score0,score1,score2)

In [ ]:
### Test Rid Reg


y_pred0 = predict_labels(w0,tx_pp)
y_pred1 = predict_labels(w1,poly_tx1_rid)
y_pred2 = predict_labels(w2,poly_tx2_rid)

#y_pred0 = np.expand_dims(y_pred0, axis=1)
y_pred1 = np.expand_dims(y_pred1, axis=1)
y_pred2 = np.expand_dims(y_pred2, axis=1)


rows = prediction.shape[0]
labels = decategorize_prediction(rows, y_pred0, y_pred1, y_pred2, indices_0, indices_1, indices_2)

labels = np.where(labels == -1, 0, labels)

score = np.invert(np.logical_xor(prediction, np.squeeze(labels)))
score.mean()

In [ ]:
poly_tx1_rid.shape

### Logistic Regression

In [21]:
# init parameters
max_iter = 1000
gamma = 0.5

weights, loss = reg_logistic_regression(y, tx,0.0001, max_iter, gamma)

Regularized Logistic Regression: Loss= 0.49750322109096384


In [22]:
ypred = predict_labels(weights,tx)
ypred = np.where(ypred == -1, 0, ypred)
ypred = np.squeeze(ypred)

(ypred == y).mean()

0.72747600000000001

#### Logistic Regression for Subsets

In [27]:
poly_tx0,_,_ = standardize(build_poly(x0, 2))
poly_tx1,_,_ = standardize(build_poly(x1, 2))
poly_tx2,_,_ = standardize(build_poly(x2, 2))



In [28]:
poly_tx0 = build_poly(x0, 2)
poly_tx1 = build_poly(x1, 2)
poly_tx2 = build_poly(x2, 2)

In [ ]:
poly_tx0[np.where(poly_tx0>1000)].shape

In [29]:
# init parameters
max_iter = 1000
gamma = 0.2
w0, loss0 = reg_logistic_regression(y0, poly_tx0,0.00001,max_iter, gamma)

/Users/erdembocugoz/Desktop/Machine Learning/MLProject1/implementations.py:229: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))


Regularized Logistic Regression: Loss= 0.3579227490534113


In [ ]:
# init parameters
poly_tx0,_,_ = standardize(build_poly(x0, 7))
max_iter = 200000
gamma = 0.2
initial_w = np.zeros((poly_tx0.shape[1], 1))
w00, loss00 = reg_logistic_regression(y0, poly_tx0,0.00001,max_iter, gamma)

In [42]:
poly_tx0 = (build_poly(x0, 2))
y_pred0 = predict_labels(w0,poly_tx0)
y_pred0 = np.where(y_pred0 == -1, 0, y_pred0)

score0 = (np.squeeze(y_pred0) == y0).mean()

score0


0.83706824937695801

In [32]:
gamma = 0.2
max_iter  = 1000
w1_log, loss1 = reg_logistic_regression(y1, poly_tx1,0.00001, max_iter, gamma)

Regularized Logistic Regression: Loss= 0.446400049846263


In [43]:
y_pred1 = predict_labels(w1_log,poly_tx1)
y_pred1 = np.where(y_pred1 == -1, 0, y_pred1)

score1 = (np.squeeze(y_pred1) == y1).mean()

score1

0.79757299081811617

In [35]:
max_iter = 1000
gamma = 0.1
w2, loss2 = reg_logistic_regression(y2, poly_tx2,0.00001, max_iter, gamma)

Regularized Logistic Regression: Loss= 0.6575174195553155


In [44]:
y_pred2 = predict_labels(w2,poly_tx2)
y_pred2 = np.where(y_pred2 == -1, 0, y_pred2)

score2 = (np.squeeze(y_pred2) == y2).mean()

score2


0.81169788952759048

In [45]:
#y_pred0 = np.expand_dims(y_pred0, axis=1)
#y_pred1 = np.expand_dims(y_pred1, axis=1)
rows = prediction.shape[0]

#y_pred2 = np.expand_dims(y_pred2, axis=1)
labels = decategorize_prediction(rows, y_pred0, y_pred1, y_pred2, indices_0, indices_1, indices_2)

labels = np.where(labels == -1, 0, labels)

score = np.invert(np.logical_xor(prediction, np.squeeze(labels)))
score.mean()

0.81745599999999996

#### Testing Logistic Regression Results

In [ ]:
### Test Log Reg

y_pred0 = predict_labels(w0,tx0)
y_pred1 = predict_labels(w1,tx1)
y_pred2 = predict_labels(w2,tx2)

rows = prediction.shape[0]
labels = decategorize_prediction(rows, y_pred0, y_pred1, y_pred2, indices_0, indices_1, indices_2)

labels = np.where(labels == -1, 0, labels)

score = prediction == np.squeeze(labels)
score.mean()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

clf0 = Ridge(alpha=0.00001)
clf1 = LogisticRegression(random_state=0, solver='sag')
scores0  = cross_val_score(clf1,poly_tx0,pred_0,cv=3)
scores0

In [ ]:

scores2  = cross_val_score(clf1,poly_tx2,pred_2,cv=3)
print(scores1,scores2)

In [ ]:
scores1  = cross_val_score(clf1,tx0,pred_0,cv=3)

In [ ]:
scores1

### Regularized Logistic Regression

In [ ]:
# Define the parameters of the algorithm.
max_iter = 1000
gamma = 0.1
lambda_ = 0.001

tx_poly = build_poly(x,2)

weights, loss = reg_logistic_regression(y, tx_poly, lambda_, max_iter, gamma)

In [ ]:
ypred = predict_labels(weights,tx)

ypred = np.squeeze(ypred)

(ypred == y).mean()
round(0.73008399999999996,4)

#### Regularized Logistic Regression for Subsets

In [ ]:
# Define the parameters of the algorithm.
max_iter = 1000
gamma = 0.9
lambda_ = 0.1

w0, loss0 = reg_logistic_regression(y0, poly_tx0, lambda_,max_iter, gamma)

In [ ]:
w1, loss1 = reg_logistic_regression(y1, tx1, lambda_,max_iter, gamma)

In [ ]:
w2, loss2 = reg_logistic_regression(y2, tx2, lambda_,max_iter, gamma)

### Submission of Results

In [ ]:
test_label,test_data,test_id_ = load_csv_data("test.csv", sub_sample=False)

# Split the data



In [ ]:
pred_0_test, pred_1_test, pred_2_test, data_0_test, data_1_test, data_2_test, indices_0_test, indices_1_test, indices_2_test = categorize_data(test_label, test_data)



In [ ]:
# Clean the data
#data_0_test,_,_ = process_data(data_0_test)
data_1_test,_,_  = process_data(data_1_test)
data_2_test,_,_  = process_data(data_2_test)



In [ ]:

# Standardize the data
y0_test, tx0_test = build_model_data(pred_0_test,data_0_test)

y1_test, tx1_test = build_model_data(pred_1_test,data_1_test)

y2_test, tx2_test = build_model_data(pred_2_test,data_2_test)
tx0_test = build_poly(data_0_test,2)
tx1_test = build_poly(data_1_test,12)
tx2_test = build_poly(data_2_test,12)

In [ ]:
y_pred0.shape

In [ ]:

# Label predictions
y_pred0 = predict_labels(w0,tx0_test)
y_pred1 = predict_labels(w1,tx1_test)
y_pred2 = predict_labels(w2,tx2_test)

#y_pred0 = np.expand_dims(y_pred0, axis=1)
y_pred1 = np.expand_dims(y_pred1, axis=1)
y_pred2 = np.expand_dims(y_pred2, axis=1)

In [ ]:

# Prepare for submission
rows = test_label.shape[0]
labels = decategorize_prediction(rows, y_pred0, y_pred1, y_pred2, indices_0_test, indices_1_test, indices_2_test)

# Create submission file


In [ ]:
labels

In [ ]:
create_csv_submission(test_id_,labels,"submission3.csv")